In [1]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json
import pandas as pd
import requests

In [4]:
load_dotenv()

True

In [5]:
client_id = os.getenv("CLIENT_ID")

In [6]:
client_secret = os.getenv("CLIENT_SECRET")

In [7]:
print(client_id, client_secret)

bf0a7a4e7fba44c3844ebceae6d06eed 86769368b8a048c69219a0923564cf59


In [8]:
##Create a function to get authorization token
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [9]:
##Get authorization token
token = get_token()
print(token)

BQCkd0JBXQb3eHpD9hby-KbvdEK1-dfuP5QiyiU3guJrKksg4YikZwhoIXSw4V29IbKMzFVPMkoLHH7N0T6GmiS4bfXbQ6DVyVN5KEW-qvgNMuJ-UmE


In [10]:
##Validate API token
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [11]:
## Create a function to look for artist name
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search/"
    headers = get_auth_header(token)
    query = f"q={artist_name}&type=artist,track&limit=1"
    
    query_url = url + "?" + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("no artists with this name exists")
        return None
    return json_result[0]

In [12]:
search_for_artist(token, "Peso Pluma")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/12GqGscKJx3aE4t07u7eVZ'},
 'followers': {'href': None, 'total': 9520689},
 'genres': ['corridos tumbados', 'sad sierreno'],
 'href': 'https://api.spotify.com/v1/artists/12GqGscKJx3aE4t07u7eVZ',
 'id': '12GqGscKJx3aE4t07u7eVZ',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb730e71d60e047f1061a9e697',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174730e71d60e047f1061a9e697',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178730e71d60e047f1061a9e697',
   'width': 160}],
 'name': 'Peso Pluma',
 'popularity': 90,
 'type': 'artist',
 'uri': 'spotify:artist:12GqGscKJx3aE4t07u7eVZ'}

In [13]:
def get_songs_by_artists(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

In [16]:
songs = get_songs_by_artists(token, "12GqGscKJx3aE4t07u7eVZ")
pd.json_normalize(songs)

,artists,disc_number,duration_ms,explicit,href,id,is_local,is_playable,name,popularity,...,album.images,album.is_playable,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,1,172797,True,https://api.spotify.com/v1/tracks/5RqSsdzTNPX1...,5RqSsdzTNPX1uzkmlHCFvK,False,True,QLONA,93,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,MAÑANA SERÁ BONITO (BICHOTA SEASON),2023-08-10,day,9,album,spotify:album:0FqAaUEyKCyUNFE1uQPZ7i,USUG12305257,https://open.spotify.com/track/5RqSsdzTNPX1uzk...
1,[{'external_urls': {'spotify': 'https://open.s...,1,212878,True,https://api.spotify.com/v1/tracks/7mXuWTczZNxG...,7mXuWTczZNxG5EDcjFEuJR,False,True,LADY GAGA,93,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,GÉNESIS,2023-06-29,day,17,album,spotify:album:4jox3ip1I39DFC2B7R5qLH,QM7282306246,https://open.spotify.com/track/7mXuWTczZNxG5ED...
2,[{'external_urls': {'spotify': 'https://open.s...,1,165671,False,https://api.spotify.com/v1/tracks/3qQbCzHBycnD...,3qQbCzHBycnDpGskqOWY0E,False,True,Ella Baila Sola,85,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,DESVELADO,2023-04-28,day,16,album,spotify:album:5aDEezKnOqyQo0qvTFhpkM,USE7D2300007,https://open.spotify.com/track/3qQbCzHBycnDpGs...
3,[{'external_urls': {'spotify': 'https://open.s...,1,234352,True,https://api.spotify.com/v1/tracks/2UW7JaomAMuX...,2UW7JaomAMuX9pZrjVpHAU,False,True,La Bebe - Remix,90,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,La Bebe (Remix),2023-03-17,day,2,album,spotify:album:6aBVGuOUEuX18rHxyDWbti,MXF152300159,https://open.spotify.com/track/2UW7JaomAMuX9pZ...
4,[{'external_urls': {'spotify': 'https://open.s...,1,209415,True,https://api.spotify.com/v1/tracks/7bPp2NmpmyhL...,7bPp2NmpmyhLJ7zWazAXMu,False,True,TULUM,89,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,GÉNESIS,2023-06-29,day,17,album,spotify:album:4jox3ip1I39DFC2B7R5qLH,QM7282325573,https://open.spotify.com/track/7bPp2NmpmyhLJ7z...
5,[{'external_urls': {'spotify': 'https://open.s...,1,187225,False,https://api.spotify.com/v1/tracks/1haJsMtoBhHf...,1haJsMtoBhHfvuM7XWuT3W,False,True,El Azul,88,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,El Azul,2023-02-10,day,1,album,spotify:album:2szyIt0ulaHnKxxyrlUCDB,QZ9QQ2300115,https://open.spotify.com/track/1haJsMtoBhHfvuM...
6,[{'external_urls': {'spotify': 'https://open.s...,1,153031,True,https://api.spotify.com/v1/tracks/1RzjXhgnzqb3...,1RzjXhgnzqb3dMa5852itx,False,True,LA PEOPLE,87,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,GÉNESIS,2023-06-29,day,17,album,spotify:album:4jox3ip1I39DFC2B7R5qLH,QM7282306250,https://open.spotify.com/track/1RzjXhgnzqb3dMa...
7,[{'external_urls': {'spotify': 'https://open.s...,1,150618,True,https://api.spotify.com/v1/tracks/2hn2zOA2XFlv...,2hn2zOA2XFlv6DSmesylrL,False,True,QUEMA,87,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,QUEMA,2023-07-13,day,1,album,spotify:album:0RdSengz6ayOfcVwH3HBry,USQX92304184,https://open.spotify.com/track/2hn2zOA2XFlv6DS...
8,[{'external_urls': {'spotify': 'https://open.s...,1,265045,True,https://api.spotify.com/v1/tracks/3wlIAXSDtlD9...,3wlIAXSDtlD9iU8ysld06Z,False,True,BIPOLAR,87,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,BIPOLAR,2023-09-07,day,1,album,spotify:album:6obRcLVFuI4x6jan7GNkYJ,QMBZ92388584,https://open.spotify.com/track/3wlIAXSDtlD9iU8...
9,[{'external_urls': {'spotify': 'https://open.s...,1,231246,True,https://api.spotify.com/v1/tracks/3HOKxuTDmNVm...,3HOKxuTDmNVmIlCIpBiD8m,False,True,LAGUNAS,87,...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,GÉNESIS,2023-06-29,day,17,album,spotify:album:4jox3ip1I39DFC2B7R5qLH,QM7282306255,https://open.spotify.com/track/3HOKxuTDmNVmIlC...


In [18]:
songs

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/790FomKkXshlbRYZFtlgla'},
     'href': 'https://api.spotify.com/v1/artists/790FomKkXshlbRYZFtlgla',
     'id': '790FomKkXshlbRYZFtlgla',
     'name': 'KAROL G',
     'type': 'artist',
     'uri': 'spotify:artist:790FomKkXshlbRYZFtlgla'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/0FqAaUEyKCyUNFE1uQPZ7i'},
   'href': 'https://api.spotify.com/v1/albums/0FqAaUEyKCyUNFE1uQPZ7i',
   'id': '0FqAaUEyKCyUNFE1uQPZ7i',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273d026bf9d7780f6a1267b4d03',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e02d026bf9d7780f6a1267b4d03',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d00004851d026bf9d7780f6a1267b4d03',
     'width': 64}],
   'is_playable': True,
   'name': 'MAÑANA SERÁ BONITO (BICHOTA SEASON)',
   'relea